### Add Water Stress Data to PostGIS RDS 

* Purpose of script: add supply, demand and water stress data from PCRGlobWB to a postGIS Relational Database
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20171116

This script requires you to set a password for your database. The script will search for the file .password in the current working directory. You can use your terminal window to create the password.

1. dfLeft = Geometries and basin info from PostGreSQL  
1. dfRight = Water Stress data from S3  


In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M11D17 UTC 13:17


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
SCRIPT_NAME = "Y2017M11D15_RH_Add_HydroBasins_postGIS_V01"

# Database settings
DATABASE_IDENTIFIER = "aqueduct30v01"
DATABASE_NAME = "database01"
TABLE_NAME = "hybasvalid01"

# Water Stress Data
INPUT_VERSION = 6
S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M10D04_RH_Threshold_WaterStress_V02/output/" 
INPUT_FILE_NAME = "Y2017M10D04_RH_Threshold_WaterStress_V%0.2d" %(INPUT_VERSION)

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

In [23]:
tableNameWaterStress = TABLE_NAME + "waterstress"

In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [4]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M10D04_RH_Threshold_WaterStress_V02/output/Y2017M10D04_RH_Threshold_WaterStress_V05.pkl to ../../../../data/Y2017M11D15_RH_Add_HydroBasins_postGIS_V01/input/Y2017M10D04_RH_Threshold_WaterStress_V05.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M10D04_RH_Threshold_WaterStress_V02/output/Y2017M10D04_RH_Threshold_WaterStress_V06.pkl to ../../../../data/Y2017M11D15_RH_Add_HydroBasins_postGIS_V01/input/Y2017M10D04_RH_Threshold_WaterStress_V06.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M10D04_RH_Threshold_WaterStress_V02/output/Y2017M10D04_RH_Threshold_WaterStress_V05.csv to ../../../../data/Y2017M11D15_RH_Add_HydroBasins_postGIS_V01/input/Y2017M10D04_RH_Threshold_WaterStress_V05.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M10D04_RH_Threshold_WaterStress_V02/output/Y2017M10D04_RH_Threshold_WaterStress_V06.csv to ../../../../data/Y2017M11D15_RH_Add_HydroBasins_postGIS_V01/input/Y2017M10D04_R

In [5]:
import os
import boto3
import botocore
from sqlalchemy import *
import pandas as pd

In [6]:
rds = boto3.client('rds')

In [7]:
F = open(".password","r")
password = F.read().splitlines()[0]
F.close()

In [8]:
response = rds.describe_db_instances(DBInstanceIdentifier="%s"%(DATABASE_IDENTIFIER))
endpoint = response["DBInstances"][0]["Endpoint"]["Address"]

In [9]:
print(endpoint)

aqueduct30v01.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com


In [10]:
engine = create_engine('postgresql://rutgerhofste:%s@%s:5432/%s' %(password,endpoint,DATABASE_NAME))

In [11]:
connection = engine.connect()

In [12]:
sql = """SELECT column_name 
FROM information_schema.columns
WHERE table_schema = 'public' 
AND table_name   = 'hybasvalid01'"""
resultAll = connection.execute(sql).fetchall()

In [13]:
columnNamesGeometries = [r[0] for r in resultAll]

In [14]:
culumnNamesAlreadyExist = columnNamesGeometries + ["hybas_id2","unnamed: 0"]

In [15]:
culumnNamesAlreadyExist.remove('geom')

In [16]:
dfRight = pd.read_pickle(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME+".pkl"))

In [17]:
dfRight.shape

(16396, 1167)

In [18]:
dfRight.columns = map(str.lower, dfRight.columns)

In [19]:
dfRight = dfRight.set_index("pfaf_id",drop=False)

In [20]:
df2 = dfRight.copy()

In [ ]:
print(columnNamesGeometries)

In [21]:
dfRight.drop(culumnNamesAlreadyExist,1,inplace=True)

In [22]:
dfRight.head()

,count_hybas06,mean_hybas06,count_area_30s_m2,mean_area_30s_m2,total_area_30s_m2,count_pdomwn_monthy2014m09,mean_pdomwn_monthy2014m09,total_volume_pdomwn_monthy2014m09,count_reducedmeanrunoff_month_y1960y2014m04,mean_reducedmeanrunoff_month_y1960y2014m04,...,lowww_month_y2014m12,arid_month_y2014m12,aridandlowww_month_y2014m12,ws_s_excl_aridandlowww_month_y2014m12,ws_s_month_y2014m12,lowww_year_y2014,arid_year_y2014,aridandlowww_year_y2014,ws_s_excl_aridandlowww_year_y2014,ws_s_year_y2014
pfaf_id,,,,,,,,,,,,,,,,,,,,,
111011,2536.0,111011.0,2536.0,743658.186761,1.885917e+09,2536.0,0.000395,745809.175699,2479.0,3.098100e-05,...,0,0,0,5.0,5.0,0,0,0,5.0,5.0
111012,3921.0,111012.0,3921.0,746186.462653,2.925797e+09,3921.0,0.000005,13455.826007,3914.0,8.585877e-06,...,1,0,0,5.0,5.0,1,0,0,5.0,5.0
111013,1194.0,111013.0,1194.0,747422.836265,8.924229e+08,1194.0,0.000013,11996.855094,1141.0,8.167370e-06,...,1,0,0,5.0,5.0,1,0,0,5.0,5.0
111014,5605.0,111014.0,5605.0,750449.270450,4.206268e+09,5605.0,0.000003,11148.519841,5605.0,3.193022e-06,...,1,0,0,5.0,5.0,1,0,0,5.0,5.0
111015,21873.0,111015.0,21873.0,758792.279231,1.659706e+10,21769.0,0.000018,306753.393803,21086.0,5.182050e-07,...,1,0,0,5.0,5.0,1,0,0,5.0,5.0


Drop Duplicate Columns from tabel

In [28]:
dfRight.dtypes

count_hybas06                                         float64
mean_hybas06                                          float64
count_area_30s_m2                                     float64
mean_area_30s_m2                                      float64
total_area_30s_m2                                     float64
count_pdomwn_monthy2014m09                            float64
mean_pdomwn_monthy2014m09                             float64
total_volume_pdomwn_monthy2014m09                     float64
count_reducedmeanrunoff_month_y1960y2014m04           float64
mean_reducedmeanrunoff_month_y1960y2014m04            float64
total_volume_reducedmeanrunoff_month_y1960y2014m04    float64
count_plivwn_yeary2014m12                             float64
mean_plivwn_yeary2014m12                              float64
total_volume_plivwn_yeary2014m12                      float64
count_pdomwn_monthy2014m10                            float64
mean_pdomwn_monthy2014m10                             float64
total_vo

In [ ]:
#columnsToDrop = ["HYBAS_ID2","HYBAS_ID","Unnamed: 0","NEXT_DOWN","NEXT_SINK","MAIN_BAS","DIST_SINK","DIST_MAIN","SUB_AREA","UP_AREA","ENDO","COAST","ORDER","SORT"]

In [25]:
dfRight["pfaf_id2"] = dfRight.index

In [29]:
dfRight.shape

(16396, 1143)

In [36]:
dfRight01 = dfRight[0:1]

In [37]:
dfRight01.shape

(1, 1143)

In [40]:
dfRight01.to_sql(tableNameWaterStress,engine,if_exists='replace', index=False,chunksize=100)

OperationalError: (psycopg2.OperationalError) row is too big: size 9168, maximum size 8160
 [SQL: 'INSERT INTO hybasvalid01waterstress (count_hybas06, mean_hybas06, count_area_30s_m2, mean_area_30s_m2, total_area_30s_m2, count_pdomwn_monthy2014m09, mean_pdomwn_monthy2014m09, total_volume_pdomwn_monthy2014m09, count_reducedmeanrunoff_month_y1960y2014m04, mean_reducedmeanrunoff_month_y1960y2014m04, total_volume_reducedmeanrunoff_month_y1960y2014m04, count_plivwn_yeary2014m12, mean_plivwn_yeary2014m12, total_volume_plivwn_yeary2014m12, count_pdomwn_monthy2014m10, mean_pdomwn_monthy2014m10, total_volume_pdomwn_monthy2014m10, count_plivww_yeary2014m12, mean_plivww_yeary2014m12, total_volume_plivww_yeary2014m12, count_runoff_monthy2014m06, mean_runoff_monthy2014m06, total_volume_runoff_monthy2014m06, count_reducedmeanrunoff_month_y1960y2014m05, mean_reducedmeanrunoff_month_y1960y2014m05, total_volume_reducedmeanrunoff_month_y1960y2014m05, count_plivww_monthy2014m08, mean_plivww_monthy2014m08